In [33]:
import pickle
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
import gzip
import shutil
# import os
import pandas as pd
from sklearn.utils import shuffle

from joblib import Parallel, delayed
import tqdm
import pyarrow
# Paths of datasets



ranked_full_game_save_path = 'C:/Users/jaspa/Grant ML/ranked_full_subfolders'
ranked_segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/ranked_segments_subfolders'
ranked_segment_save_path_bulk = 'C:/Users/jaspa/Grant ML/ranked_segments_bulk'

public_full_game_save_path = 'C:/Users/jaspa/Grant ML/public_full_subfolders'
public_segment_save_path_subfolder = 'C:/Users/jaspa/Grant ML/public_segments_subfolders'
public_segment_save_path_bulk = 'C:/Users/jaspa/Grant ML/public_segments_bulk'

List all slp game input arrays we want to process. Make a dataframe with the data.

In [34]:
# def list_files(directory):
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             yield os.path.join(root, file)

# def parse_file_path(file_path):
#     parts = file_path.split('\\')
#     # Extracting necessary details from the file path
#     file_name = parts[-1]
#     character = parts[-2]
#     opponent = parts[-3]
#     return {
#         'file': file_name,
#         'path': '\\'.join(parts[:-1]),
#         'character': character,
#         'opponent': opponent,
#         'game_length': 0,  # Placeholder values
#         'segment_shift': 0,
#         'num_segments': 0
#     }

# def generate_dataframe_from_folders(folders):
#     files_data = []
#     for folder in folders:
#         for file_path in list_files(folder):
#             files_data.append(parse_file_path(file_path))
#     return pd.DataFrame(files_data)

# ranked_full_game_save_path = 'C:/Users/jaspa/Grant ML/ranked_full_subfolders'
# public_full_game_save_path = 'C:/Users/jaspa/Grant ML/public_full_subfolders'
# folders = [ranked_full_game_save_path, public_full_game_save_path]

# df = generate_dataframe_from_folders(folders)
# print(df.head())  # Display the first few rows to check

Use Joblib to open the game arrays and get the length of the game in frames. Save the data frame. About 40 seconds.

In [35]:
# def get_array_length(file_path):
#     # Update this function based on how you want to handle the file reading,
#     # For gzipped numpy files, you would do something like this:
#     with gzip.open(file_path, 'rb') as f:
#         arr = np.load(f)
#         return arr.shape[1]

# def process_row(row):
#     # Call the get_array_length function with the path of the file
#     length = get_array_length(row['path'] + '\\' + row['file'])
#     return length


# num_cores = -1  # Or however many cores you want to use
# lengths = Parallel(n_jobs=num_cores,verbose = 0)(delayed(process_row)(row) for index, row in tqdm.tqdm(df.iterrows()))
# df['game_length'] = lengths

# df.to_feather('C:/Users/jaspa/Grant ML/slp/data/path_info_df.feather')


# print(df.head())  # To check the updated DataFrame


In [36]:
# Define the path to your feather file
file_path = 'C:/Users/jaspa/Grant ML/slp/data/path_info_df.feather'

# Load the feather file
df = pd.read_feather(file_path)
df.head()

,file,path,character,opponent,game_length,segment_shift,num_segments
0,107729__1__CAPTAIN_FALCON__BOWSER.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,10935,0,0
1,1122__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,11244,0,0
2,113580__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,7487,0,0
3,116020__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,12676,0,0
4,13522__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,15657,0,0


Set the number of segments you want to use (same for each character). Check what the maximum power of two you need to shift each segment by to get the right number of segments. Can shift by as little as 64 frames.

In [37]:
min_segments = 250000
segment_length = 1024
for character in df['character'].unique():
    game_lengths = df.loc[df['character'] == character, 'game_length']
    for i in range(10):
        segment_shift = 2 ** (10-i)
        num_segments = 0
        for game_length in game_lengths:
            num_segments += (game_length - segment_length) // segment_shift
        if num_segments > min_segments:
            break
    print(character, ' ',segment_shift)
    # Correct way to set values to avoid SettingWithCopyWarning
    df.loc[df['character'] == character, 'segment_shift'] = segment_shift

df.head() 
# print(df.head())

CAPTAIN_FALCON   1024
DONKEY_KONG   64
DR_MARIO   128
FALCO   1024
FOX   1024
GAME_AND_WATCH   64
GANONDORF   256
ICE_CLIMBERS   256
JIGGLYPUFF   512
KIRBY   16
LINK   64
LUIGI   128
MARIO   64
MARTH   1024
MEWTWO   64
NESS   64
PEACH   512
PICHU   8
PIKACHU   128
ROY   32
SAMUS   256
SHEIK   1024
YOSHI   128
YOUNG_LINK   64
ZELDA   32
BOWSER   32


,file,path,character,opponent,game_length,segment_shift,num_segments
0,107729__1__CAPTAIN_FALCON__BOWSER.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,10935,1024,0
1,1122__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,11244,1024,0
2,113580__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,7487,1024,0
3,116020__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,12676,1024,0
4,13522__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,15657,1024,0


Check how many segments you will get from each game.

In [38]:
df['num_segments'] = (df['game_length']- segment_length) // df['segment_shift']
df.head()

,file,path,character,opponent,game_length,segment_shift,num_segments
0,107729__1__CAPTAIN_FALCON__BOWSER.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,10935,1024,9
1,1122__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,11244,1024,9
2,113580__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,7487,1024,6
3,116020__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,12676,1024,11
4,13522__0__BOWSER__CAPTAIN_FALCON.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,CAPTAIN_FALCON,BOWSER,15657,1024,14


See how many extra segments you have for each character.

In [39]:
# min_segments = 20000
# segment_length = 1024
for character in df['character'].unique():
    character_df = df.loc[df['character'] == character]
    print(character, ': ', sum(character_df['num_segments'])-min_segments)


CAPTAIN_FALCON :  67909
DONKEY_KONG :  93664
DR_MARIO :  77506
FALCO :  391333
FOX :  586085
GAME_AND_WATCH :  44888
GANONDORF :  10160
ICE_CLIMBERS :  72141
JIGGLYPUFF :  95916
KIRBY :  171972
LINK :  195064
LUIGI :  219267
MARIO :  49038
MARTH :  194374
MEWTWO :  110591
NESS :  225165
PEACH :  147862
PICHU :  82087
PIKACHU :  112329
ROY :  169726
SAMUS :  220073
SHEIK :  32434
YOSHI :  191234
YOUNG_LINK :  5084
ZELDA :  222705
BOWSER :  92948


Randomize the games from each character. Cound how many segments you are going to get for each character. Split the games up so that 15% of the segments end up in the test set and the validation and 70% of the segments end up in the train set. No segments from the same game will end up in different data sets.

In [40]:


# Initialize empty DataFrames for test, validation, and training sets
test_df = pd.DataFrame()
val_df = pd.DataFrame()
train_df = pd.DataFrame()

# min_segments = 20000
# target_segments_for_test_and_val = min_segments * 0.15

for character in df['character'].unique():
    character_df = df.loc[df['character'] == character].sample(frac=1).reset_index(drop=True)  # Shuffle the rows
    sum_segments = 0
    test_idx = 0
    val_idx = 0
    
    #split the games of each character by .15, .15, and .7
    target_segments_for_test_and_val = sum(character_df['num_segments']) * 0.15
    # Determine the split indices for test and validation sets
    for i, row in character_df.iterrows():
        sum_segments += row['num_segments']
        if sum_segments >= target_segments_for_test_and_val:
            if test_idx == 0:  # First batch for test set
                test_idx = i
                sum_segments = 0  # Reset sum for validation set calculation
            elif val_idx == 0:  # Next batch for validation set
                val_idx = i
                break
    
    # Split the character_df into test, val, and train based on the indices
    test_rows = character_df.iloc[:test_idx+1]
    val_rows = character_df.iloc[test_idx+1:val_idx+1]
    train_rows = character_df.iloc[val_idx+1:]

    # Append to respective DataFrames
    test_df = pd.concat([test_df, test_rows], ignore_index=True)
    val_df = pd.concat([val_df, val_rows], ignore_index=True)
    train_df = pd.concat([train_df, train_rows], ignore_index=True)
    
print(test_df.shape)
print(val_df.shape)
print(train_df.shape)

print(train_df.head())

(62757, 7)
(62916, 7)
(293406, 7)
                                      file  \
0  243747__2__CAPTAIN_FALCON__FALCO.npy.gz   
1  233785__1__CAPTAIN_FALCON__SAMUS.npy.gz   
2  218872__3__CAPTAIN_FALCON__MARTH.npy.gz   
3  287089__3__CAPTAIN_FALCON__SHEIK.npy.gz   
4    281765__2__FOX__CAPTAIN_FALCON.npy.gz   

                                                path       character opponent  \
0  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    FALCO   
1  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    SAMUS   
2  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    MARTH   
3  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON    SHEIK   
4  C:/Users/jaspa/Grant ML/public_full_subfolders...  CAPTAIN_FALCON      FOX   

   game_length  segment_shift  num_segments  
0         6821           1024             5  
1         8401           1024             7  
2         9924           1024             8  
3         9112    

Expand the data frames so that there is one path for each segment.

In [41]:
# import pandas as pd
# import numpy as np

def expand_df_vectorized(df):
    # Calculate the repeat count for each row based on 'num_segments'
    repeats = df['num_segments'].values
    
    # Repeat each index according to its corresponding 'num_segments' value
    index_repeated = np.repeat(df.index, repeats)
    
    # Create a new DataFrame by repeating rows
    df_repeated = df.loc[index_repeated].reset_index(drop=True)
    
    # Create a 'segment_index' column that counts up for each group of repeated rows
    segment_indices = np.concatenate([np.arange(n) for n in repeats])
    
    # Assign 'segment_index' to the repeated DataFrame
    df_repeated['segment_index'] = segment_indices
    
    # Optionally, drop the 'num_segments' column if it's no longer needed
    df_repeated = df_repeated.drop(columns=['num_segments'])
    
    return df_repeated


# # Example usage:
expanded_test_df = expand_df_vectorized(test_df)
expanded_val_df = expand_df_vectorized(val_df)
expanded_train_df = expand_df_vectorized(train_df)

print(expanded_test_df.shape)
print(expanded_val_df.shape)
print(expanded_train_df.shape)
print(expanded_test_df.head())

(1559815, 7)
(1559648, 7)
(7262092, 7)
                                     file  \
0  114162__0__LINK__CAPTAIN_FALCON.npy.gz   
1  114162__0__LINK__CAPTAIN_FALCON.npy.gz   
2  114162__0__LINK__CAPTAIN_FALCON.npy.gz   
3  114162__0__LINK__CAPTAIN_FALCON.npy.gz   
4  114162__0__LINK__CAPTAIN_FALCON.npy.gz   

                                                path       character opponent  \
0  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON     LINK   
1  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON     LINK   
2  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON     LINK   
3  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON     LINK   
4  C:/Users/jaspa/Grant ML/ranked_full_subfolders...  CAPTAIN_FALCON     LINK   

   game_length  segment_shift  segment_index  
0         6525           1024              0  
1         6525           1024              1  
2         6525           1024              2  
3         6525 

Randomly sample the right number of segments to create each data set.  Encode the character.

In [47]:
def sample_rows_per_character(df, proportion, min_segments, encoder,characters):
    rows_per_character = int(min_segments * proportion)
    sampled_df = pd.DataFrame()
    
    for character in characters:
        character_df = df[df['character'] == character]
        sampled_rows = character_df.sample(n=min(rows_per_character, len(character_df)), random_state=1)
        sampled_df = pd.concat([sampled_df, sampled_rows], ignore_index=True)
    
    # Add the 'labels' column using the fitted encoder
    sampled_df['labels'] = encoder.transform(sampled_df['character'])
    
    return sampled_df


# Assuming 'characters' is your array of unique character names
characters =  [
                'FOX', 
                # 'FALCO', 
                # 'MARTH', 
                'SHEIK', 
                # 'CAPTAIN_FALCON', 
                # 'PEACH', 
                # 'JIGGLYPUFF', 
                # 'SAMUS', 
                # 'ICE_CLIMBERS', 
                # 'GANONDORF', 
                # 'YOSHI', 
                # 'LUIGI', 
                # 'PIKACHU', 
                # 'DR_MARIO', 
                # 'NESS', 
                # 'LINK', 
                # 'MEWTWO', 
                # 'GAME_AND_WATCH', 
                # 'DONKEY_KONG', 
                # 'YOUNG_LINK', 
                # 'MARIO', 
                # 'ROY', 
                # 'BOWSER', 
                # 'ZELDA', 
                # 'KIRBY', 
                # 'PICHU'
                ]

# Initialize and fit the LabelEncoder
encoder = LabelEncoder()
encoder.fit(characters)

sampled_test_df = sample_rows_per_character(expanded_test_df, 0.15, min_segments, encoder,characters)
sampled_val_df = sample_rows_per_character(expanded_val_df, 0.15, min_segments, encoder,characters)
sampled_train_df = sample_rows_per_character(expanded_train_df, 0.70, min_segments, encoder,characters)


print(sampled_test_df.shape)
print(sampled_val_df.shape)
print(sampled_train_df.shape)

sampled_test_df

(75000, 8)
(75000, 8)
(350000, 8)


,file,path,character,opponent,game_length,segment_shift,segment_index,labels
0,96052__1__FOX__SHEIK.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,FOX,SHEIK,14264,1024,2,0
1,207688__1__FOX__FOX.npy.gz,C:/Users/jaspa/Grant ML/public_full_subfolders...,FOX,FOX,13020,1024,1,0
2,111232__0__SHEIK__FOX.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,FOX,SHEIK,11992,1024,5,0
3,37207__0__FOX__FOX.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,FOX,FOX,10824,1024,6,0
4,67532__0__JIGGLYPUFF__FOX.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,FOX,JIGGLYPUFF,10607,1024,4,0
...,...,...,...,...,...,...,...,...
74995,249959__3__SHEIK__FOX.npy.gz,C:/Users/jaspa/Grant ML/public_full_subfolders...,SHEIK,FOX,9129,1024,1,1
74996,288191__1__SHEIK__FALCO.npy.gz,C:/Users/jaspa/Grant ML/public_full_subfolders...,SHEIK,FALCO,13154,1024,8,1
74997,77500__0__FOX__SHEIK.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,SHEIK,FOX,9185,1024,6,1
74998,26445__1__SHEIK__JIGGLYPUFF.npy.gz,C:/Users/jaspa/Grant ML/ranked_full_subfolders...,SHEIK,JIGGLYPUFF,14292,1024,3,1


In [43]:
sampled_test_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_test_df.feather')
sampled_val_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_val_df.feather')
sampled_train_df.to_feather('C:/Users/jaspa/Grant ML/slp/data/sample_train_df.feather')


In [44]:
df['character'].unique()

array(['CAPTAIN_FALCON', 'DONKEY_KONG', 'DR_MARIO', 'FALCO', 'FOX',
       'GAME_AND_WATCH', 'GANONDORF', 'ICE_CLIMBERS', 'JIGGLYPUFF',
       'KIRBY', 'LINK', 'LUIGI', 'MARIO', 'MARTH', 'MEWTWO', 'NESS',
       'PEACH', 'PICHU', 'PIKACHU', 'ROY', 'SAMUS', 'SHEIK', 'YOSHI',
       'YOUNG_LINK', 'ZELDA', 'BOWSER'], dtype=object)